# Trending Wikipedia

1. Get trending Wikipedia articles from yesterday
2. Use LLM for suggestsions as to why each article is trending
3. Build HTML page to display each article and why it is trending

In [ ]:
 # !pip install requests ollama

API call for the featured feed shows different for today than it does for any previous days.

This is written for previous days only

In [ ]:
import requests
import datetime

today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)

date_to_query = yesterday
url = 'https://api.wikimedia.org/feed/v1/wikipedia/en/featured/' + date_to_query.strftime('%Y/%m/%d')


response = requests.get(url)
featured_feed = response.json()
print(f"API call: {url}")
print(f"Retrieved Wikipedia top article statistics for {date_to_query}")

### Save API response to file

In [ ]:
import os
import json

# Ensure the 'featured-feed' folder exists
file_directory = "data"
os.makedirs(file_directory, exist_ok=True)

# Define the filename based on the date
base_file_name = date_to_query.strftime('%Y-%m-%d')
file_path = f'{file_directory}/{base_file_name}.json'

# Save to JSON file (overwrite if it already exists)
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(featured_feed, file, indent=4, ensure_ascii=False)

print(f'Saved Wikipedia response to {file_path}')

#### On top ten articles
1. Prompt Ollama to generate a reason this article might be trending based on the extract
2. Create new list with only vital information

In [ ]:
import ollama

articles_with_reasons = []


for item in featured_feed['mostread']['articles'][:10]:
    title = item['title']
    views = item['views']
    link = item['content_urls']['desktop']['page']
    extract = item['extract']
    thumbnail = item.get('thumbnail', {}).get('source', None)
    print(f"Analyzing {title}")

    prompt = f"Act as a professional news summarizer. Based on your knowledge of {title} and the following extract. In 1-2 sentences, explain why the {title} article might be trending on Wikipedia on #{date_to_query}:\n\n{extract}"


    # Call the Ollama chat API with the model and the prompt
    response = ollama.chat(model='phi3', messages=[
    {
        'role': 'user',
        'content': prompt,
    },
    ])

    print(f"trendingreason: {response['message']['content']}")
    
    article={
        'title': title,
        'views': views,
        'link': link,
        'thumbnail': thumbnail,
        'extract': extract,
        'trendingreason': response['message']['content']
    }



    articles_with_reasons.append(article)


### Save vital information to new file

In [ ]:
file_path = f'{file_directory}/{base_file_name}-trending-reasons.json'

# Save to JSON file (overwrite if it already exists)
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(articles_with_reasons, file, indent=4, ensure_ascii=False)

print(f'articles_with_reasons saved to {file_path}')

#### Build HTML Page to display the top 10 list complete with thumbnails and the reason generated by Ollama

In [7]:
# Start building the HTML
html_list = "<ol>\n"

# Iterate through the data
for item in articles_with_reasons:
    title = item['title']
    link = item['link']
    thumbnail = item['thumbnail']
    trendingreason = item['trendingreason']
    views = item['views']
    extract = item['extract']

    # Handle null thumbnail
    if thumbnail:
        thumbnail_html = f'<img src="{thumbnail}" alt="Thumbnail for {title}"/><br>'
    else:
        thumbnail_html = '<p><em>No thumbnail available</em></p>'
    
    # Create a list item for each entry
    html_list += f"""
    <li>
        <h2>
          <a href="{link}" target="_blank">{title}</a><br>
        </h2>
        {thumbnail_html}
        <strong>Views:</strong> {views}<br>
        <strong>Extract:</strong> {extract}<br>
        <strong>Reason for Trending:</strong> {trendingreason}
    </li>\n
    """

# Close the HTML list
html_list += "</ol>"

# Save to html file (overwrite if it already exists)
file_path = f'{file_directory}/{base_file_name}.html'

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(html_list)

# Display the HTML in the notebook
from IPython.display import display, HTML
display(HTML(html_list))